In [ ]:
#@title 🎧 Download Narration Audio & Play Introduction
import os as _os
if not _os.path.exists("/content/narration"):
    !pip install -q gdown
    import gdown
    gdown.download(id="11gcJ8_8aATzq7q6k1ExNgH2pUCTE2at1", output="/content/narration.zip", quiet=False)
    !unzip -q /content/narration.zip -d /content/narration
    !rm /content/narration.zip
    print(f"Loaded {len(_os.listdir('/content/narration'))} narration segments")
else:
    print("Narration audio already loaded.")

from IPython.display import Audio, display
display(Audio("/content/narration/00_intro.mp3"))

In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

# 🚀 Building a Complete Context Engine: The Four Strategies in Action

*Part 4 of the Vizuara series on Context Engineering for LLMs*
*Estimated time: 35 minutes*

# 🤖 AI Teaching Assistant

Need help with this notebook? Open the **AI Teaching Assistant** — it has already read this entire notebook and can help with concepts, code, and exercises.

**[👉 Open AI Teaching Assistant](https://course-creator-brown.vercel.app/courses/context-engineering-for-llms/practice/4/assistant)**

*Tip: Open it in a separate tab and work through this notebook side-by-side.*


In [ ]:
# Setup and imports
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from pathlib import Path
from collections import Counter
import re
import os
import tempfile

%matplotlib inline

np.random.seed(42)

print("Welcome to Part 4: Building a Complete Context Engine!")
print("No API keys needed — everything runs locally.\n")
print("Libraries loaded: json, numpy, matplotlib, pathlib, sklearn (later)")

In [ ]:
#@title 🎧 Listen: Why It Matters
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_why_it_matters.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 1. Why Does This Matter?

Modern LLMs have context windows of 128K, 200K, even 1M tokens. That sounds like a lot — until you realize that a real-world agent might need to juggle:

- A **system prompt** (2K tokens)
- **Conversation history** (50K tokens after an hour of coding)
- **Retrieved documents** (30K tokens from a codebase search)
- **Tool results** (20K tokens of grep output)
- **Persistent memory** (project conventions, user preferences)

That is already more than 100K tokens, and we have not even left room for the model to *think* and *respond* (which needs 30-50K tokens of output budget).

**Context engineering** is the discipline of managing this finite resource intelligently. Lance Martin (LangChain) and Anthropic codified four canonical strategies:

| Strategy | Core Idea | Analogy |
|----------|-----------|---------|
| **Write** | Persist important info outside the window | Saving phone numbers in contacts instead of memorizing |
| **Select** | Retrieve just-in-time, not everything upfront | Looking up one book from the library vs carrying all books |
| **Compress** | Summarize and compact, don't truncate blindly | Meeting notes instead of full transcript |
| **Isolate** | Split context across sub-agents | Delegating research to team members who report back summaries |

In this notebook, you will build each strategy as a standalone class, combine them into a full `ContextEngine`, and measure exactly how much each strategy saves.

In [ ]:
#@title 🎧 Listen: Overflow Simulation
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_overflow_simulation.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

In [ ]:
# Let us see the problem concretely: what happens without context engineering?

# Simulate a realistic agent context
system_prompt = "You are a senior software engineer helping with a Python web application." * 10
conversation_history = [
    f"Turn {i}: {'User asks about feature implementation.' if i % 2 == 0 else 'Assistant provides detailed code with explanations, examples, and best practices.'}" * (3 if i % 2 == 1 else 1)
    for i in range(80)
]
tool_results = [
    f"grep result {i}: " + "def function_name(param1, param2, param3):\n    " * 20 + f"# match {i}"
    for i in range(15)
]
project_memory = [
    "Convention: use snake_case for Python functions",
    "Build command: pytest tests/ -v --cov",
    "Database: PostgreSQL 15, schema in migrations/",
    "Style: Black formatter, 88 char line length",
    "Deploy: docker-compose up --build",
]

def estimate_tokens(text):
    """Rough token estimate: ~4 characters per token."""
    return len(text) // 4

total_system = estimate_tokens(system_prompt)
total_history = sum(estimate_tokens(msg) for msg in conversation_history)
total_tools = sum(estimate_tokens(r) for r in tool_results)
total_memory = sum(estimate_tokens(m) for m in project_memory)
total_all = total_system + total_history + total_tools + total_memory

context_limit = 128000
output_reserved = 35000
available = context_limit - output_reserved

print("=" * 60)
print("  THE CONTEXT BUDGET PROBLEM")
print("=" * 60)
print(f"\n  Context window:        {context_limit:>8,} tokens")
print(f"  Reserved for output:   {output_reserved:>8,} tokens")
print(f"  Available for input:   {available:>8,} tokens")
print(f"\n  System prompt:         {total_system:>8,} tokens")
print(f"  Conversation history:  {total_history:>8,} tokens")
print(f"  Tool results:          {total_tools:>8,} tokens")
print(f"  Project memory:        {total_memory:>8,} tokens")
print(f"  ─────────────────────────────────")
print(f"  TOTAL needed:          {total_all:>8,} tokens")
print(f"\n  Over budget by:        {max(0, total_all - available):>8,} tokens")
print(f"  Utilization:           {total_all / available * 100:>7.1f}%")

if total_all > available:
    print("\n  ⚠️  We are OVER BUDGET. Without context engineering,")
    print("     we would have to truncate — losing important information.")
else:
    print("\n  We fit, but barely. As the session grows, we will overflow.")

In [ ]:
#@title 🎧 Listen: Building Intuition
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_building_intuition.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 2. Building Intuition

### The Four Strategies as Everyday Actions

Before we write any code, let us build a mental model for each strategy using everyday analogies.

**Strategy 1 — Write (Persist Outside the Window)**

Imagine you are studying for an exam. You cannot memorize every single fact from a 500-page textbook. Instead, you write flashcards. The knowledge lives *outside your head* (outside the context window) but is available whenever you need it.

Real-world example: Claude Code's `CLAUDE.md` file. Instead of re-explaining project conventions every message, the agent writes them to a persistent file and reads them back when needed.

**Strategy 2 — Select (Retrieve Just-in-Time)**

You are in a library with 10,000 books. You do not carry all of them to your desk. You check the catalog, find the 3 books relevant to your current research question, and fetch only those.

Real-world example: Cursor's `@`-references let users manually select which files to include. Claude Code's `glob`/`grep` tools let the agent discover context lazily.

**Strategy 3 — Compress (Summarize and Compact)**

After a 2-hour meeting, you do not transcribe every word. You write bullet-point notes capturing the key decisions, action items, and unresolved questions. The 50-page transcript becomes 1 page of notes.

Real-world example: Claude Code's auto-compact triggers at 95% capacity — it summarizes conversation history, clears verbose tool outputs, and keeps only the conclusions.

**Strategy 4 — Isolate (Sub-Agent Architectures)**

You are a project manager. Instead of personally reading every document, you assign each team member a specific task: "Alice, summarize the backend architecture. Bob, review the test coverage. Carol, check the deployment config." Each person works with a clean, focused context and reports back a concise summary.

Real-world example: Anthropic's multi-agent researcher splits research across sub-agents, each with its own context window, and collects condensed results.

In [ ]:
#@title 🎧 Listen: Reflection
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_reflection.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### 🤔 Think About This

Before building anything, predict:

1. Which strategy do you think saves the **most** tokens? Why?
2. Which strategy is **cheapest** to implement (no ML, no embeddings)?
3. Could you use *only one* strategy and still have a good system? Or do you need all four?

Write down your predictions. We will revisit them at the end.

In [ ]:
#@title 🎧 Listen: Intuition Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_intuition_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

In [ ]:
# Quick intuition builder: how much does each strategy typically save?

strategies = ['Write\n(Persist)', 'Select\n(Retrieve)', 'Compress\n(Summarize)', 'Isolate\n(Sub-agents)']
typical_savings_pct = [15, 40, 50, 30]  # rough estimates from production systems
typical_effort = [1, 3, 2, 4]  # implementation complexity (1=easy, 4=hard)

fig, axes = plt.subplots(1, 2, figsize=(13, 5))
colors = ['#4CAF50', '#2196F3', '#FF9800', '#9C27B0']

# Token savings
bars1 = axes[0].bar(strategies, typical_savings_pct, color=colors,
                     edgecolor='white', linewidth=2, width=0.6)
for bar, val in zip(bars1, typical_savings_pct):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                 f'{val}%', ha='center', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Typical Token Savings (%)', fontsize=12)
axes[0].set_title('Token Savings by Strategy', fontsize=13, fontweight='bold')
axes[0].set_ylim(0, 65)
axes[0].grid(True, alpha=0.3, axis='y')

# Implementation effort
bars2 = axes[1].bar(strategies, typical_effort, color=colors,
                     edgecolor='white', linewidth=2, width=0.6)
effort_labels = ['Easy', 'Medium', 'Medium', 'Hard']
for bar, val, lbl in zip(bars2, typical_effort, effort_labels):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
                 lbl, ha='center', fontsize=11, fontweight='bold')
axes[1].set_ylabel('Implementation Complexity', fontsize=12)
axes[1].set_title('Effort to Implement', fontsize=13, fontweight='bold')
axes[1].set_ylim(0, 5.5)
axes[1].set_yticks([1, 2, 3, 4])
axes[1].set_yticklabels(['Easy', 'Medium', 'Hard', 'Very Hard'])
axes[1].grid(True, alpha=0.3, axis='y')

plt.suptitle('The Four Context Engineering Strategies — Overview',
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("Each strategy targets a different part of the context budget.")
print("The real power comes from combining all four.")

In [ ]:
#@title 🎧 Listen: Mathematics
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/06_mathematics.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 3. The Mathematics

### 3.1 The Context Budget Equation

A context window of $C$ tokens must accommodate:

$$C = T_{\text{system}} + T_{\text{memory}} + T_{\text{history}} + T_{\text{retrieved}} + T_{\text{query}} + T_{\text{output}}$$

We control the input side. The output budget $T_{\text{output}}$ is reserved (typically 25-35% of $C$). Our job is to fit everything else into the remaining budget:

$$T_{\text{available}} = C - T_{\text{output}}$$

### 3.2 Strategy Formulas

**Write (Persist):** Instead of repeating $n$ facts of average size $s$ tokens every turn, we write them once and load on demand. Savings per turn:

$$\Delta_{\text{write}} = n \cdot s \cdot (1 - \frac{1}{k})$$

where $k$ is the number of turns between re-reads. If you write a fact and read it every 10 turns instead of repeating it every turn, you save 90%.

**Select (Retrieve):** From a corpus of $N$ documents of average size $d$ tokens, we retrieve only $k \ll N$:

$$\Delta_{\text{select}} = (N - k) \cdot d$$

**Compress (Summarize):** A conversation of $H$ tokens compresses to $H \cdot r$ tokens, where $r$ is the compression ratio (typically 0.1–0.3):

$$\Delta_{\text{compress}} = H \cdot (1 - r)$$

**Isolate (Sub-agents):** Instead of one agent with context $C$, we have $n$ sub-agents each with context $C/n$, returning summaries of $s$ tokens:

$$T_{\text{lead}} = T_{\text{system}} + T_{\text{query}} + n \cdot s$$

In [ ]:
#@title 🎧 Listen: Math Computation
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/07_math_computation.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

In [ ]:
# Let us compute exact savings for our scenario

C = 128000
T_output = 35000
T_available = C - T_output

# Current usage (from above)
T_system = total_system
T_history = total_history
T_tools = total_tools
T_memory_tokens = total_memory
T_query = 100  # user's current question

T_total = T_system + T_history + T_tools + T_memory_tokens + T_query

# Strategy savings (theoretical)
# Write: memory facts don't need to be repeated in history
write_savings = T_memory_tokens * 0  # already small; real savings come from not repeating in history
# But more importantly, without Write, those facts bloat every conversation turn
# Assume 20 turns where conventions were restated = ~200 tokens each
write_savings = 20 * 50  # 20 repetitions of ~50 tokens each

# Select: retrieve 5 of 15 tool results instead of all 15
select_k = 5
select_savings = sum(estimate_tokens(r) for r in tool_results[select_k:])

# Compress: compress history to 30% of original
compress_ratio = 0.3
compress_savings = int(T_history * (1 - compress_ratio))

# Isolate: each sub-agent gets focused context, returns 500-token summary
# instead of dumping everything into one window
isolate_sub_summaries = 3 * 500  # 3 sub-agents, 500 tokens each
isolate_savings = max(0, T_tools - isolate_sub_summaries)

print("=" * 65)
print("  THEORETICAL TOKEN SAVINGS PER STRATEGY")
print("=" * 65)
print(f"\n  Original total:     {T_total:>8,} tokens")
print(f"  Available budget:   {T_available:>8,} tokens")
print(f"\n  Write savings:      {write_savings:>8,} tokens  (avoid repeating facts)")
print(f"  Select savings:     {select_savings:>8,} tokens  (fetch {select_k} of {len(tool_results)} results)")
print(f"  Compress savings:   {compress_savings:>8,} tokens  (compress history to {compress_ratio:.0%})")
print(f"  Isolate savings:    {isolate_savings:>8,} tokens  (sub-agent summaries)")
print(f"\n  Combined savings:   {write_savings + select_savings + compress_savings + isolate_savings:>8,} tokens")

after_all = T_total - (write_savings + select_savings + compress_savings + isolate_savings)
print(f"  After all 4:        {after_all:>8,} tokens")
print(f"  Fits in budget?     {'Yes' if after_all <= T_available else 'No'}")

In [ ]:
#@title 🎧 Listen: Strategy1 Write Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/08_strategy1_write_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 4. Let's Build It — The Four Strategies

### 4.1 Strategy 1: Write (MemoryStore)

The simplest strategy — and often the most overlooked. Persist important facts to a JSON file so they survive across sessions and do not need to be repeated in every message.

In [ ]:
class MemoryStore:
    """
    Strategy 1: Write — Persist facts outside the context window.

    Like saving phone numbers in your contacts instead of memorizing them.
    Claude Code uses CLAUDE.md for exactly this purpose.
    """

    def __init__(self, memory_path=None):
        if memory_path is None:
            # Use a temp file for Colab
            self.memory_path = Path(tempfile.mkdtemp()) / "memory.json"
        else:
            self.memory_path = Path(memory_path)
        self._ensure_file()

    def _ensure_file(self):
        """Create memory file if it does not exist."""
        if not self.memory_path.exists():
            self.memory_path.write_text(json.dumps({"facts": {}, "metadata": {"created": "session_start", "version": 1}}, indent=2))

    def save(self, key, value, category="general"):
        """
        Save a fact to persistent memory.

        Args:
            key: identifier for the fact (e.g., "build_command")
            value: the fact itself (e.g., "npm run build")
            category: grouping (e.g., "project", "user_preference")
        """
        data = json.loads(self.memory_path.read_text())
        data["facts"][key] = {
            "value": value,
            "category": category,
            "access_count": 0
        }
        self.memory_path.write_text(json.dumps(data, indent=2))

    def load(self):
        """Load all memories from persistent storage."""
        data = json.loads(self.memory_path.read_text())
        # Track access counts
        for key in data["facts"]:
            data["facts"][key]["access_count"] += 1
        self.memory_path.write_text(json.dumps(data, indent=2))
        return data["facts"]

    def search(self, query, top_k=5):
        """
        Find relevant memories using keyword matching.
        Simple but effective — no embeddings needed.
        """
        query_words = set(query.lower().split())
        facts = self.load()

        scored = []
        for key, entry in facts.items():
            # Score based on word overlap with key and value
            text_words = set((key + " " + entry["value"]).lower().split())
            overlap = len(query_words & text_words)
            if overlap > 0:
                scored.append((key, entry["value"], overlap))

        scored.sort(key=lambda x: x[2], reverse=True)
        return scored[:top_k]

    def get_token_cost(self):
        """How many tokens does our memory store cost to include?"""
        facts = json.loads(self.memory_path.read_text())["facts"]
        text = "\n".join(f"{k}: {v['value']}" for k, v in facts.items())
        return estimate_tokens(text)

    def format_for_context(self):
        """Format memories as a context block."""
        facts = json.loads(self.memory_path.read_text())["facts"]
        if not facts:
            return ""
        lines = []
        for key, entry in facts.items():
            lines.append(f"- {key}: {entry['value']}")
        return "<memory>\n" + "\n".join(lines) + "\n</memory>"

In [ ]:
#@title 🎧 Listen: Write Demo
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/09_write_demo.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

In [ ]:
# Demo: Using the MemoryStore

memory = MemoryStore()

# Save project conventions (like Claude Code's CLAUDE.md)
memory.save("build_command", "pytest tests/ -v --cov", category="project")
memory.save("formatter", "Black, 88 char line length", category="project")
memory.save("database", "PostgreSQL 15, migrations in alembic/", category="project")
memory.save("deploy_command", "docker-compose up --build", category="project")
memory.save("naming_convention", "snake_case for Python, camelCase for JS", category="style")
memory.save("test_framework", "pytest with conftest.py fixtures", category="project")
memory.save("api_style", "REST with FastAPI, OpenAPI docs at /docs", category="project")
memory.save("error_handling", "Use custom exceptions in app/errors.py", category="project")

print("Saved 8 project facts to persistent memory.\n")

# Search for relevant memories
print("Search: 'how to run tests'")
results = memory.search("how to run tests")
for key, value, score in results:
    print(f"  [{score}] {key}: {value}")

print("\nSearch: 'database schema'")
results = memory.search("database schema")
for key, value, score in results:
    print(f"  [{score}] {key}: {value}")

print(f"\nToken cost of including all memories: {memory.get_token_cost()} tokens")
print(f"\nFormatted context block:")
print(memory.format_for_context())

### Why Write Matters

The savings from Write are not just about the memory file itself (which is small). The real savings come from **not repeating** these facts in conversation. Without a persistent memory, a user might say "remember, we use Black formatter" three times in a session — each time adding tokens to the history. With Write, the fact is stored once and loaded on demand.

In [ ]:
# Demonstrate the savings: repetition vs persistence

# Without Write: user repeats conventions in conversation
repeated_facts = [
    "Remember, we use Black formatter with 88 char lines",
    "The test command is pytest tests/ -v --cov",
    "Our database is PostgreSQL 15",
    "We deploy with docker-compose up --build",
    "Use snake_case for Python function names",
]

# Simulate 5 repetitions over a conversation
repetitions = 5
tokens_without_write = sum(estimate_tokens(f) for f in repeated_facts) * repetitions
tokens_with_write = memory.get_token_cost()  # loaded once from file

print(f"Without Write (facts repeated {repetitions}x): {tokens_without_write:,} tokens")
print(f"With Write (loaded once from file):           {tokens_with_write:,} tokens")
print(f"Savings:                                      {tokens_without_write - tokens_with_write:,} tokens ({(1 - tokens_with_write/tokens_without_write)*100:.0f}%)")

In [ ]:
#@title 🎧 Listen: Strategy2 Select
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/10_strategy2_select.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### 4.2 Strategy 2: Select (DocumentSelector)

Do not load everything upfront. Maintain a lightweight index and fetch full content only when the agent actually needs it.

In [ ]:
class DocumentSelector:
    """
    Strategy 2: Select — Retrieve just-in-time, not everything upfront.

    Like checking the library catalog instead of carrying all books.
    Cursor uses @-references; Claude Code uses glob/grep for lazy discovery.
    """

    def __init__(self):
        self.index = {}  # doc_id -> metadata (small)
        self.store = {}  # doc_id -> full content (large, loaded lazily)

    def register(self, doc_id, summary, content, tags=None):
        """
        Register a document. Stores metadata in the index (cheap)
        and full content in the store (expensive, loaded only on demand).
        """
        self.index[doc_id] = {
            "summary": summary,
            "tags": tags or [],
            "token_count": estimate_tokens(content),
        }
        self.store[doc_id] = content

    def select(self, query, top_k=5):
        """
        Select the most relevant document IDs WITHOUT loading full content.
        Uses keyword matching against summaries and tags.
        Returns list of (doc_id, relevance_score, metadata).
        """
        query_words = set(query.lower().split())
        scored = []

        for doc_id, meta in self.index.items():
            text = (meta["summary"] + " " + " ".join(meta["tags"])).lower()
            text_words = set(text.split())
            overlap = len(query_words & text_words)
            if overlap > 0:
                scored.append((doc_id, overlap, meta))

        scored.sort(key=lambda x: x[1], reverse=True)
        return scored[:top_k]

    def fetch(self, doc_id):
        """
        Fetch the full content of a document (just-in-time loading).
        This is the expensive operation — only call when truly needed.
        """
        if doc_id in self.store:
            return self.store[doc_id]
        return None

    def eager_load_cost(self):
        """Token cost of loading ALL documents (the naive approach)."""
        return sum(estimate_tokens(content) for content in self.store.values())

    def lazy_load_cost(self, selected_ids):
        """Token cost of loading only selected documents."""
        return sum(estimate_tokens(self.store[doc_id])
                   for doc_id in selected_ids if doc_id in self.store)

    def index_cost(self):
        """Token cost of the lightweight index (always loaded)."""
        text = "\n".join(f"{did}: {m['summary']}"
                         for did, m in self.index.items())
        return estimate_tokens(text)

In [ ]:
#@title 🎧 Listen: Select Demo
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/11_select_demo.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

In [ ]:
# Demo: Simulate a codebase with 20 files

selector = DocumentSelector()

# Register simulated codebase files
files = [
    ("auth/login.py", "User authentication and login flow with JWT tokens",
     "import jwt\nfrom fastapi import APIRouter\n\nrouter = APIRouter()\n\n" + "def login(username, password):\n    # Validate credentials against database\n    user = db.query(User).filter_by(username=username).first()\n    if not user or not verify_password(password, user.hashed_password):\n        raise HTTPException(status_code=401)\n    token = jwt.encode({'sub': user.id}, SECRET_KEY)\n    return {'access_token': token}\n" * 5,
     ["auth", "login", "jwt", "security"]),

    ("auth/permissions.py", "Role-based access control and permission decorators",
     "from functools import wraps\n\ndef require_role(role):\n    def decorator(func):\n        @wraps(func)\n        def wrapper(*args, **kwargs):\n            if current_user.role != role:\n                raise PermissionError\n            return func(*args, **kwargs)\n        return wrapper\n    return decorator\n" * 4,
     ["auth", "permissions", "rbac", "decorator"]),

    ("models/user.py", "SQLAlchemy User model with profile fields",
     "from sqlalchemy import Column, Integer, String, DateTime\nfrom app.database import Base\n\nclass User(Base):\n    __tablename__ = 'users'\n    id = Column(Integer, primary_key=True)\n    username = Column(String(50), unique=True)\n    email = Column(String(120), unique=True)\n    hashed_password = Column(String(256))\n    role = Column(String(20), default='user')\n    created_at = Column(DateTime)\n" * 3,
     ["model", "user", "database", "sqlalchemy"]),

    ("models/product.py", "Product catalog model with pricing and inventory",
     "class Product(Base):\n    __tablename__ = 'products'\n    id = Column(Integer, primary_key=True)\n    name = Column(String(200))\n    description = Column(Text)\n    price = Column(Numeric(10, 2))\n    inventory_count = Column(Integer, default=0)\n    category_id = Column(Integer, ForeignKey('categories.id'))\n" * 4,
     ["model", "product", "pricing", "inventory"]),

    ("api/routes.py", "Main API router with all endpoint registrations",
     "from fastapi import FastAPI\napp = FastAPI(title='MyApp')\n\napp.include_router(auth_router, prefix='/auth')\napp.include_router(product_router, prefix='/products')\napp.include_router(order_router, prefix='/orders')\napp.include_router(user_router, prefix='/users')\n" * 3,
     ["api", "routes", "fastapi", "endpoints"]),

    ("api/products.py", "Product CRUD endpoints with search and filtering",
     "from fastapi import APIRouter, Query\n\nrouter = APIRouter()\n\n@router.get('/')\ndef list_products(category: str = None, min_price: float = None):\n    query = db.query(Product)\n    if category: query = query.filter_by(category=category)\n    if min_price: query = query.filter(Product.price >= min_price)\n    return query.all()\n\n@router.post('/')\ndef create_product(product: ProductCreate):\n    db_product = Product(**product.dict())\n    db.add(db_product)\n    db.commit()\n    return db_product\n" * 3,
     ["api", "products", "crud", "search", "filtering"]),

    ("tests/test_auth.py", "Authentication unit tests",
     "import pytest\nfrom app.auth.login import login\n\ndef test_valid_login():\n    result = login('admin', 'password123')\n    assert 'access_token' in result\n\ndef test_invalid_password():\n    with pytest.raises(HTTPException):\n        login('admin', 'wrong_password')\n" * 4,
     ["test", "auth", "pytest", "login"]),

    ("tests/test_products.py", "Product endpoint integration tests",
     "import pytest\nfrom fastapi.testclient import TestClient\n\ndef test_list_products(client):\n    response = client.get('/products/')\n    assert response.status_code == 200\n    assert isinstance(response.json(), list)\n\ndef test_create_product(client, admin_token):\n    response = client.post('/products/', json={'name': 'Widget', 'price': 9.99})\n    assert response.status_code == 201\n" * 3,
     ["test", "products", "integration", "fastapi"]),

    ("config/settings.py", "Application configuration with environment variables",
     "from pydantic import BaseSettings\n\nclass Settings(BaseSettings):\n    DATABASE_URL: str = 'postgresql://localhost/myapp'\n    SECRET_KEY: str = 'change-me'\n    DEBUG: bool = False\n    CORS_ORIGINS: list = ['http://localhost:3000']\n    class Config:\n        env_file = '.env'\n\nsettings = Settings()\n" * 2,
     ["config", "settings", "environment", "pydantic"]),

    ("config/database.py", "Database connection and session management",
     "from sqlalchemy import create_engine\nfrom sqlalchemy.orm import sessionmaker\n\nengine = create_engine(settings.DATABASE_URL)\nSessionLocal = sessionmaker(bind=engine)\n\ndef get_db():\n    db = SessionLocal()\n    try:\n        yield db\n    finally:\n        db.close()\n" * 3,
     ["database", "connection", "sqlalchemy", "session"]),

    ("middleware/cors.py", "CORS middleware configuration",
     "from fastapi.middleware.cors import CORSMiddleware\n\ndef setup_cors(app):\n    app.add_middleware(CORSMiddleware, allow_origins=settings.CORS_ORIGINS)\n" * 2,
     ["middleware", "cors", "security"]),

    ("middleware/logging.py", "Request logging middleware with timing",
     "import time\nimport logging\n\nlogger = logging.getLogger(__name__)\n\nclass LoggingMiddleware:\n    async def __call__(self, request, call_next):\n        start = time.time()\n        response = await call_next(request)\n        duration = time.time() - start\n        logger.info(f'{request.method} {request.url} {response.status_code} {duration:.3f}s')\n        return response\n" * 3,
     ["middleware", "logging", "timing", "request"]),

    ("utils/email.py", "Email sending utility with templates",
     "import smtplib\nfrom email.mime.text import MIMEText\n\ndef send_email(to, subject, body):\n    msg = MIMEText(body)\n    msg['Subject'] = subject\n    msg['To'] = to\n    with smtplib.SMTP(settings.SMTP_HOST) as server:\n        server.send_message(msg)\n" * 3,
     ["email", "utility", "smtp", "notification"]),

    ("utils/pagination.py", "Pagination helper for list endpoints",
     "def paginate(query, page=1, per_page=20):\n    total = query.count()\n    items = query.offset((page-1)*per_page).limit(per_page).all()\n    return {'items': items, 'total': total, 'page': page, 'pages': (total+per_page-1)//per_page}\n" * 2,
     ["pagination", "utility", "list", "query"]),

    ("migrations/001_initial.py", "Initial database migration creating core tables",
     "def upgrade():\n    op.create_table('users', sa.Column('id', sa.Integer, primary_key=True), sa.Column('username', sa.String(50)))\n    op.create_table('products', sa.Column('id', sa.Integer, primary_key=True), sa.Column('name', sa.String(200)))\n" * 4,
     ["migration", "database", "schema", "alembic"]),
]

for filepath, summary, content, tags in files:
    selector.register(filepath, summary, content, tags)

print(f"Registered {len(files)} files in the document index.\n")
print(f"Index cost (always in context):  {selector.index_cost():>6,} tokens")
print(f"Eager load (all files):          {selector.eager_load_cost():>6,} tokens")
print(f"Ratio:                           {selector.index_cost() / selector.eager_load_cost() * 100:.1f}% of full load")

In [ ]:
# Now simulate a query: "fix the login authentication bug"

query = "fix the login authentication bug"
print(f"Query: '{query}'\n")

# Step 1: Select relevant documents (cheap — only uses index)
selected = selector.select(query, top_k=3)
print("Step 1 — Select (from index only):")
for doc_id, score, meta in selected:
    print(f"  [{score}] {doc_id}: {meta['summary']} ({meta['token_count']} tokens)")

# Step 2: Fetch full content (expensive — only for selected docs)
selected_ids = [doc_id for doc_id, _, _ in selected]
print(f"\nStep 2 — Fetch (just-in-time):")
for doc_id in selected_ids:
    content = selector.fetch(doc_id)
    print(f"  Loaded {doc_id}: {estimate_tokens(content)} tokens")

# Compare costs
eager_cost = selector.eager_load_cost()
lazy_cost = selector.lazy_load_cost(selected_ids) + selector.index_cost()
savings = eager_cost - lazy_cost

print(f"\n{'='*50}")
print(f"  Eager loading (all {len(files)} files): {eager_cost:>6,} tokens")
print(f"  Lazy loading ({len(selected_ids)} files + index): {lazy_cost:>6,} tokens")
print(f"  Savings:                        {savings:>6,} tokens ({savings/eager_cost*100:.0f}%)")

In [ ]:
#@title 🎧 Listen: Strategy3 Compress
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/12_strategy3_compress.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### 4.3 Strategy 3: Compress (ContextCompressor)

When approaching the context limit, compress intelligently. Keep high-signal information (architectural decisions, unresolved bugs), discard redundancy.

In [ ]:
class ContextCompressor:
    """
    Strategy 3: Compress — Summarize and compact, don't truncate blindly.

    Like writing meeting notes instead of keeping the full transcript.
    Claude Code auto-compacts at 95% capacity.
    """

    def __init__(self):
        pass

    def compress_history(self, messages, budget_tokens):
        """
        Truncate from oldest, keeping the most recent messages
        that fit within the token budget.

        This is the simplest compression: just drop old messages.
        """
        compressed = []
        tokens_used = 0

        for msg in reversed(messages):
            msg_tokens = estimate_tokens(msg)
            if tokens_used + msg_tokens > budget_tokens:
                break
            compressed.insert(0, msg)
            tokens_used += msg_tokens

        return compressed, tokens_used

    def clear_tool_results(self, messages, max_result_tokens=100):
        """
        Strip verbose tool outputs but keep conclusions.
        Simulates Claude Code's behavior of clearing tool output
        after processing.

        Before: "grep found 500 lines of code: [500 lines]..."
        After:  "grep found 500 lines of code: [truncated, 500 lines matched]"
        """
        cleaned = []
        tokens_saved = 0

        for msg in messages:
            if msg.startswith("TOOL_RESULT:"):
                # Keep first line (the summary) and truncate the rest
                lines = msg.split("\n")
                summary_line = lines[0]
                original_tokens = estimate_tokens(msg)

                if original_tokens > max_result_tokens:
                    n_lines = len(lines) - 1
                    truncated = f"{summary_line}\n[truncated: {n_lines} lines, {original_tokens} tokens of output]"
                    cleaned.append(truncated)
                    tokens_saved += original_tokens - estimate_tokens(truncated)
                else:
                    cleaned.append(msg)
            else:
                cleaned.append(msg)

        return cleaned, tokens_saved

    def format_compressed(self, messages):
        """Format compressed messages as a context block."""
        return "<history>\n" + "\n".join(messages) + "\n</history>"

In [ ]:
# Demo: Compress a realistic conversation history

# Simulate a 40-turn conversation with tool results
conversation = []
for i in range(40):
    if i % 2 == 0:
        conversation.append(f"USER: How do I implement feature {i//2 + 1}?")
    elif i % 5 == 1:
        # Every 5th assistant turn includes a verbose tool result
        tool_output = f"TOOL_RESULT: grep search for 'feature_{i//2}':\n" + \
            "\n".join([f"  src/module_{j}.py:L{j*10}: def feature_{i//2}_handler(request):" +
                      f"    # Implementation of feature {i//2} with detailed logic and error handling"
                      for j in range(30)])
        conversation.append(tool_output)
    else:
        conversation.append(f"ASSISTANT: Here is how to implement feature {i//2 + 1}. " +
                          "First, create the handler function. Then add the route. " +
                          "Finally, write tests to verify the behavior. " +
                          "Make sure to handle edge cases like invalid input and rate limiting.")

compressor = ContextCompressor()

# Measure original
original_tokens = sum(estimate_tokens(msg) for msg in conversation)
print(f"Original conversation: {len(conversation)} messages, {original_tokens:,} tokens\n")

# Strategy A: Simple truncation (keep newest)
budget = original_tokens // 3
compressed_a, tokens_a = compressor.compress_history(conversation, budget)
print(f"Strategy A — Truncate from oldest:")
print(f"  Budget: {budget:,} tokens")
print(f"  Kept {len(compressed_a)} of {len(conversation)} messages ({tokens_a:,} tokens)")
print(f"  Savings: {original_tokens - tokens_a:,} tokens ({(1 - tokens_a/original_tokens)*100:.0f}%)\n")

# Strategy B: Clear tool results
cleaned_b, saved_b = compressor.clear_tool_results(conversation)
tokens_b = sum(estimate_tokens(msg) for msg in cleaned_b)
print(f"Strategy B — Clear tool results:")
print(f"  Cleaned {len(cleaned_b)} messages ({tokens_b:,} tokens)")
print(f"  Savings: {saved_b:,} tokens ({saved_b/original_tokens*100:.0f}%)\n")

# Strategy C: Both combined
cleaned_c, saved_c = compressor.clear_tool_results(conversation)
compressed_c, tokens_c = compressor.compress_history(cleaned_c, budget)
print(f"Strategy C — Clear tools + truncate:")
print(f"  Kept {len(compressed_c)} messages ({tokens_c:,} tokens)")
print(f"  Total savings: {original_tokens - tokens_c:,} tokens ({(1 - tokens_c/original_tokens)*100:.0f}%)")

In [ ]:
#@title 🎧 Listen: Compression Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/13_compression_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### 📊 Visualization Checkpoint 1: Compression Comparison

In [ ]:
# Visualize the three compression strategies

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

strategies_comp = ['Original', 'Truncate\n(oldest)', 'Clear\ntools', 'Both\ncombined']
token_counts = [original_tokens, tokens_a, tokens_b, tokens_c]
colors_comp = ['#E53935', '#FFA726', '#42A5F5', '#66BB6A']

# Bar chart of token counts
axes[0].bar(strategies_comp, token_counts, color=colors_comp,
            edgecolor='white', linewidth=2)
for i, (s, t) in enumerate(zip(strategies_comp, token_counts)):
    axes[0].text(i, t + 50, f'{t:,}', ha='center', fontsize=10, fontweight='bold')
axes[0].set_ylabel('Tokens', fontsize=12)
axes[0].set_title('Token Count by Strategy', fontsize=13, fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='y')

# Savings percentages
savings_pcts = [0, (1-tokens_a/original_tokens)*100,
                (1-tokens_b/original_tokens)*100,
                (1-tokens_c/original_tokens)*100]
axes[1].bar(strategies_comp, savings_pcts, color=colors_comp,
            edgecolor='white', linewidth=2)
for i, pct in enumerate(savings_pcts):
    axes[1].text(i, pct + 1, f'{pct:.0f}%', ha='center', fontsize=10, fontweight='bold')
axes[1].set_ylabel('Savings (%)', fontsize=12)
axes[1].set_title('Token Savings', fontsize=13, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')

# Messages retained
msg_counts = [len(conversation), len(compressed_a), len(cleaned_b), len(compressed_c)]
axes[2].bar(strategies_comp, msg_counts, color=colors_comp,
            edgecolor='white', linewidth=2)
for i, mc in enumerate(msg_counts):
    axes[2].text(i, mc + 0.5, str(mc), ha='center', fontsize=10, fontweight='bold')
axes[2].set_ylabel('Messages', fontsize=12)
axes[2].set_title('Messages Retained', fontsize=13, fontweight='bold')
axes[2].grid(True, alpha=0.3, axis='y')

plt.suptitle('Compression Strategy Comparison', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 Listen: Todo1
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/14_todo1.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 5. 🔧 Your Turn — TODO Sections

### TODO 1: Implement `summarize_history` using TF-IDF

The `compress_history` method above is crude — it just drops old messages. A smarter approach extracts the **most important sentences** across the entire history using TF-IDF (Term Frequency-Inverse Document Frequency).

TF-IDF scores each sentence by how "distinctive" its words are. Sentences with rare, topic-specific words score higher than sentences with common words like "the" or "is".

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

def summarize_history(messages, max_sentences=5):
    """
    Extract the most important sentences from conversation history
    using TF-IDF scoring.

    How it works:
    1. Split all messages into individual sentences
    2. Compute TF-IDF vectors for each sentence
    3. Score each sentence by the sum of its TF-IDF values
       (sentences with rare, distinctive words score higher)
    4. Return the top-k highest-scoring sentences

    Args:
        messages: list of conversation messages (strings)
        max_sentences: number of sentences to keep

    Returns:
        list of the most important sentences (in original order)
    """
    # ============ TODO ============
    # Step 1: Split all messages into sentences
    #   - Join all messages, then split on '. ' or '.\n' or '?' or '!'
    #   - Filter out empty strings and very short sentences (< 10 chars)
    #
    # Step 2: Compute TF-IDF
    #   - Create a TfidfVectorizer()
    #   - Fit and transform the sentences
    #   - For each sentence, compute its score as the sum of its TF-IDF values
    #     (hint: use .toarray() and .sum(axis=1))
    #
    # Step 3: Select top sentences
    #   - Get indices of top max_sentences by score
    #   - Sort those indices to preserve original order
    #   - Return the corresponding sentences
    #
    # YOUR CODE BELOW:
    # ==============================

    pass  # Replace with your implementation


# ✅ Verification: Run this to test your implementation
test_messages = [
    "The authentication system uses JWT tokens with RS256 signing.",
    "I went to the store today. It was nice weather outside.",
    "Critical bug: the refresh token rotation is broken in production.",
    "The user model has fields for username, email, and hashed_password.",
    "Maybe we should consider updating the dependencies. That would be good.",
    "The database migration 003 adds a foreign key constraint on orders.user_id.",
    "Everything looks fine to me. Nothing special going on here.",
    "PostgreSQL connection pooling is configured with max_connections=20.",
    "The API rate limiter uses a sliding window algorithm with Redis backend.",
    "I think the weather will be good tomorrow. Should we go for a walk?",
]

result = summarize_history(test_messages, max_sentences=4)

if result is not None:
    print("Your extracted summary:")
    for i, sentence in enumerate(result):
        print(f"  {i+1}. {sentence}")

    # Check that it picked technical sentences over filler
    technical_keywords = ['JWT', 'token', 'database', 'PostgreSQL', 'API', 'migration', 'bug']
    technical_count = sum(1 for s in result
                         if any(kw.lower() in s.lower() for kw in technical_keywords))
    print(f"\nTechnical sentences selected: {technical_count}/{len(result)}")
    if technical_count >= 3:
        print("Excellent! Your TF-IDF correctly prioritizes technical content.")
    else:
        print("Hmm, try adjusting — TF-IDF should favor rare, specific words.")
else:
    print("TODO not yet implemented. Complete the function above!")

In [ ]:
#@title 🎧 Listen: Todo1 Solution
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/15_todo1_solution.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

Here is the reference solution (collapse this cell after attempting the TODO):

In [ ]:
# Reference solution for summarize_history

def summarize_history_solution(messages, max_sentences=5):
    """Reference solution: TF-IDF extractive summarization."""
    # Step 1: Split into sentences
    all_text = " ".join(messages)
    raw_sentences = re.split(r'(?<=[.!?])\s+', all_text)
    sentences = [s.strip() for s in raw_sentences if len(s.strip()) >= 10]

    if len(sentences) <= max_sentences:
        return sentences

    # Step 2: TF-IDF scoring
    vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
    tfidf_matrix = vectorizer.fit_transform(sentences)
    scores = tfidf_matrix.toarray().sum(axis=1)

    # Step 3: Select top sentences in original order
    top_indices = np.argsort(scores)[::-1][:max_sentences]
    top_indices_sorted = sorted(top_indices)

    return [sentences[i] for i in top_indices_sorted]


# Verify the reference solution
result_ref = summarize_history_solution(test_messages, max_sentences=4)
print("Reference solution output:")
for i, sentence in enumerate(result_ref):
    print(f"  {i+1}. {sentence}")

original_tokens_test = sum(estimate_tokens(m) for m in test_messages)
summary_tokens_test = sum(estimate_tokens(s) for s in result_ref)
print(f"\nOriginal: {original_tokens_test} tokens -> Summary: {summary_tokens_test} tokens")
print(f"Compression ratio: {summary_tokens_test/original_tokens_test:.1%}")

In [ ]:
#@title 🎧 Listen: Strategy4 Isolate
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/16_strategy4_isolate.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### 4.4 Strategy 4: Isolate (SubAgentManager)

Split context across multiple agents with clean, focused windows. Each sub-agent dives deep into one area and returns a condensed summary.

In [ ]:
class SubAgentManager:
    """
    Strategy 4: Isolate — Sub-agent architectures with separate context windows.

    Like delegating research to team members who report back summaries.
    Anthropic's multi-agent researcher outperformed single-agent on SWE-bench.
    """

    def __init__(self, max_summary_tokens=500):
        self.max_summary_tokens = max_summary_tokens
        self.sub_agents = []
        self.results = []

    def dispatch(self, task, context, agent_id=None):
        """
        Give a sub-agent a focused task with limited context.
        In production, this would call an LLM. Here we simulate.

        Args:
            task: what the sub-agent should do
            context: the focused context for this sub-agent
            agent_id: optional identifier

        Returns:
            dict with agent metadata
        """
        agent_id = agent_id or f"agent_{len(self.sub_agents)}"
        context_tokens = estimate_tokens(context)

        agent_info = {
            "agent_id": agent_id,
            "task": task,
            "context_tokens": context_tokens,
            "status": "dispatched"
        }
        self.sub_agents.append(agent_info)
        return agent_info

    def simulate_execution(self, context):
        """
        Simulate a sub-agent processing its context and producing a summary.
        In production, this calls an LLM. Here we use extractive summarization.
        """
        # Extract key lines (simulating what an LLM would summarize)
        lines = context.split("\n")
        # Keep lines with code-like patterns (def, class, import, return)
        key_patterns = ['def ', 'class ', 'import ', 'return ', '@', 'raise ',
                        'Column(', 'router.', 'assert ']
        key_lines = [l.strip() for l in lines
                     if any(p in l for p in key_patterns) and len(l.strip()) > 5]

        # Deduplicate and limit
        seen = set()
        unique_lines = []
        for line in key_lines:
            if line not in seen:
                seen.add(line)
                unique_lines.append(line)

        # Truncate to fit summary budget
        summary_lines = []
        tokens = 0
        for line in unique_lines:
            line_tokens = estimate_tokens(line)
            if tokens + line_tokens > self.max_summary_tokens:
                break
            summary_lines.append(line)
            tokens += line_tokens

        return "\n".join(summary_lines) if summary_lines else "No key findings."

    def collect_results(self):
        """
        Gather condensed results from all sub-agents.
        Each result is a short summary — much smaller than the original context.
        """
        collected = []
        for agent in self.sub_agents:
            collected.append({
                "agent_id": agent["agent_id"],
                "task": agent["task"],
                "context_tokens": agent["context_tokens"],
                "status": "completed"
            })
        return collected

    def format_for_lead_agent(self, summaries):
        """
        Format sub-agent summaries for the lead agent's context.
        The lead agent sees compact summaries, not raw documents.
        """
        blocks = []
        for agent_id, summary in summaries:
            blocks.append(f"<sub_agent id='{agent_id}'>\n{summary}\n</sub_agent>")
        return "\n\n".join(blocks)

In [ ]:
# Demo: Split a research task across 3 sub-agents

manager = SubAgentManager(max_summary_tokens=300)

# The lead agent needs to answer: "How does authentication work in this codebase?"
# Instead of loading all 15 files, we dispatch 3 focused sub-agents.

tasks = [
    ("auth_specialist", "Analyze authentication flow and security mechanisms",
     ["auth/login.py", "auth/permissions.py"]),
    ("data_specialist", "Analyze user data model and database schema",
     ["models/user.py", "config/database.py", "migrations/001_initial.py"]),
    ("test_specialist", "Review test coverage for authentication",
     ["tests/test_auth.py"]),
]

print("Dispatching sub-agents:\n")
total_sub_context = 0
summaries = []

for agent_id, task, file_ids in tasks:
    # Each sub-agent gets ONLY its relevant files
    context_parts = [selector.fetch(fid) for fid in file_ids if selector.fetch(fid)]
    sub_context = "\n\n".join(context_parts)
    sub_tokens = estimate_tokens(sub_context)
    total_sub_context += sub_tokens

    manager.dispatch(task, sub_context, agent_id)

    # Simulate execution
    summary = manager.simulate_execution(sub_context)
    summary_tokens = estimate_tokens(summary)
    summaries.append((agent_id, summary))

    print(f"  {agent_id}:")
    print(f"    Task: {task}")
    print(f"    Files: {file_ids}")
    print(f"    Context: {sub_tokens:,} tokens")
    print(f"    Summary: {summary_tokens} tokens")
    print(f"    Compression: {summary_tokens/sub_tokens*100:.0f}%\n")

# What the lead agent actually sees
lead_context = manager.format_for_lead_agent(summaries)
lead_tokens = estimate_tokens(lead_context)

# Compare with the naive approach (loading all files)
naive_tokens = selector.eager_load_cost()

print("=" * 55)
print(f"  Naive (load all files):    {naive_tokens:>6,} tokens")
print(f"  Isolation (3 summaries):   {lead_tokens:>6,} tokens")
print(f"  Savings:                   {naive_tokens - lead_tokens:>6,} tokens ({(1 - lead_tokens/naive_tokens)*100:.0f}%)")
print("=" * 55)

In [ ]:
#@title 🎧 Listen: Todo2
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/17_todo2.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### TODO 2: Implement `dispatch_to_subagents`

Now it is your turn. Implement a function that takes a set of tasks and documents, automatically splits them across sub-agents with isolated contexts, and collects their condensed results.

In [ ]:
def dispatch_to_subagents(tasks, documents, max_summary_tokens=300):
    """
    Split research across N sub-agents, each with isolated context.

    Args:
        tasks: list of dicts, each with:
            - 'name': sub-agent identifier
            - 'description': what this agent should investigate
            - 'doc_ids': list of document IDs this agent should see
        documents: dict mapping doc_id -> full content string
        max_summary_tokens: max tokens per sub-agent summary

    Returns:
        dict with:
            - 'summaries': list of (agent_name, summary_text) tuples
            - 'total_input_tokens': total tokens fed to all sub-agents
            - 'total_output_tokens': total tokens in all summaries
            - 'compression_ratio': output/input ratio
            - 'per_agent': list of dicts with per-agent stats

    How to implement:
    1. For each task, gather the documents specified by doc_ids
    2. Concatenate those documents as the sub-agent's context
    3. Generate an extractive summary (keep lines matching code patterns)
    4. Truncate summary to max_summary_tokens
    5. Collect all summaries and compute statistics
    """
    # ============ TODO ============
    # For each task:
    #   1. Gather documents: join documents[doc_id] for each doc_id in task['doc_ids']
    #   2. Count tokens in the gathered context
    #   3. Generate extractive summary:
    #      - Split context into lines
    #      - Keep lines containing: 'def ', 'class ', 'import ', 'return ',
    #        '@', 'raise ', 'Column(', 'assert '
    #      - Deduplicate
    #      - Truncate to max_summary_tokens
    #   4. Store (task['name'], summary) in summaries list
    #   5. Track per-agent stats: name, input_tokens, output_tokens
    #
    # After all tasks:
    #   - Compute total_input_tokens, total_output_tokens, compression_ratio
    #   - Return the results dict
    #
    # YOUR CODE BELOW:
    # ==============================

    pass  # Replace with your implementation


# ✅ Verification
test_tasks = [
    {'name': 'auth_agent', 'description': 'Analyze auth',
     'doc_ids': ['auth/login.py', 'auth/permissions.py']},
    {'name': 'model_agent', 'description': 'Analyze models',
     'doc_ids': ['models/user.py', 'models/product.py']},
]

test_docs = {fid: selector.fetch(fid) for fid in
             ['auth/login.py', 'auth/permissions.py', 'models/user.py', 'models/product.py']}

result = dispatch_to_subagents(test_tasks, test_docs)

if result is not None:
    print("Your sub-agent dispatch results:")
    print(f"  Total input tokens:  {result['total_input_tokens']:,}")
    print(f"  Total output tokens: {result['total_output_tokens']:,}")
    print(f"  Compression ratio:   {result['compression_ratio']:.1%}")
    print(f"\n  Per-agent breakdown:")
    for agent in result['per_agent']:
        print(f"    {agent['name']}: {agent['input_tokens']} -> {agent['output_tokens']} tokens")
    print(f"\n  Summaries:")
    for name, summary in result['summaries']:
        print(f"    [{name}]: {summary[:80]}...")
else:
    print("TODO not yet implemented. Complete the function above!")

In [ ]:
#@title 🎧 Listen: Todo2 Solution
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/18_todo2_solution.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

Here is the reference solution:

In [ ]:
# Reference solution for dispatch_to_subagents

def dispatch_to_subagents_solution(tasks, documents, max_summary_tokens=300):
    """Reference solution: dispatch tasks to isolated sub-agents."""
    summaries = []
    per_agent = []
    total_input = 0
    total_output = 0

    key_patterns = ['def ', 'class ', 'import ', 'return ', '@', 'raise ',
                    'Column(', 'assert ']

    for task in tasks:
        # Step 1: Gather documents
        context_parts = [documents[did] for did in task['doc_ids'] if did in documents]
        context = "\n\n".join(context_parts)
        input_tokens = estimate_tokens(context)
        total_input += input_tokens

        # Step 2: Extractive summary
        lines = context.split("\n")
        key_lines = [l.strip() for l in lines
                     if any(p in l for p in key_patterns) and len(l.strip()) > 5]

        # Deduplicate
        seen = set()
        unique = []
        for line in key_lines:
            if line not in seen:
                seen.add(line)
                unique.append(line)

        # Truncate to budget
        summary_lines = []
        tokens = 0
        for line in unique:
            lt = estimate_tokens(line)
            if tokens + lt > max_summary_tokens:
                break
            summary_lines.append(line)
            tokens += lt

        summary = "\n".join(summary_lines) if summary_lines else "No key findings."
        output_tokens = estimate_tokens(summary)
        total_output += output_tokens

        summaries.append((task['name'], summary))
        per_agent.append({
            'name': task['name'],
            'input_tokens': input_tokens,
            'output_tokens': output_tokens
        })

    compression = total_output / total_input if total_input > 0 else 0

    return {
        'summaries': summaries,
        'total_input_tokens': total_input,
        'total_output_tokens': total_output,
        'compression_ratio': compression,
        'per_agent': per_agent
    }


# Verify reference solution
result_ref = dispatch_to_subagents_solution(test_tasks, test_docs)
print("Reference solution:")
print(f"  Total input:  {result_ref['total_input_tokens']:,} tokens")
print(f"  Total output: {result_ref['total_output_tokens']:,} tokens")
print(f"  Compression:  {result_ref['compression_ratio']:.1%}")
for agent in result_ref['per_agent']:
    print(f"  {agent['name']}: {agent['input_tokens']} -> {agent['output_tokens']} tokens")

In [ ]:
#@title 🎧 Listen: Complete Engine
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/19_complete_engine.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 5 (continued). The Complete ContextEngine

Now we combine all four strategies into a single engine that manages the entire context budget.

In [ ]:
class ContextEngine:
    """
    The Complete Context Engine — all four strategies working together.

    This is the production pattern used (in various forms) by Claude Code,
    Cursor, Devin, and other frontier agent systems.

    Architecture:
        1. Write:    MemoryStore persists facts outside the window
        2. Select:   DocumentSelector retrieves just-in-time
        3. Compress: ContextCompressor summarizes and compacts
        4. Isolate:  SubAgentManager splits across focused agents
    """

    def __init__(self, max_tokens=128000, reserved_for_output=35000):
        self.max_tokens = max_tokens
        self.reserved = reserved_for_output
        self.available = max_tokens - reserved_for_output

        # Initialize all four strategy components
        self.memory = MemoryStore()
        self.selector = DocumentSelector()
        self.compressor = ContextCompressor()
        self.sub_agent_mgr = SubAgentManager(max_summary_tokens=500)

        # Budget allocation (percentage of available tokens)
        self.budget_allocation = {
            "system":   0.10,  # 10% for system prompt
            "memory":   0.05,  # 5% for persistent memory
            "history":  0.30,  # 30% for conversation history
            "retrieved": 0.45, # 45% for retrieved documents
            "query":    0.10,  # 10% for the current query
        }

    def _compute_budgets(self):
        """Compute token budgets for each context section."""
        return {k: int(self.available * v)
                for k, v in self.budget_allocation.items()}

    def assemble(self, query, system_prompt, history, doc_query=None):
        """
        Assemble the full context using all four strategies.

        Returns:
            dict with context string, token breakdown, and metadata
        """
        budgets = self._compute_budgets()

        # --- Strategy 1: Write (load persistent memory) ---
        memory_block = self.memory.format_for_context()
        memory_tokens = estimate_tokens(memory_block)

        # --- Strategy 2: Select (retrieve relevant documents) ---
        retrieved_docs = []
        retrieved_tokens = 0
        if doc_query and self.selector.index:
            selected = self.selector.select(doc_query, top_k=5)
            for doc_id, score, meta in selected:
                content = self.selector.fetch(doc_id)
                if content:
                    doc_tokens = estimate_tokens(content)
                    if retrieved_tokens + doc_tokens <= budgets["retrieved"]:
                        retrieved_docs.append((doc_id, content, score))
                        retrieved_tokens += doc_tokens

        # --- Strategy 3: Compress (fit history into budget) ---
        # First, clear tool results
        cleaned_history, _ = self.compressor.clear_tool_results(history)
        # Then truncate to fit budget
        compressed_history, history_tokens = self.compressor.compress_history(
            cleaned_history, budgets["history"]
        )

        # --- Strategy 4: Isolate (not used during assembly, but available) ---
        # Sub-agent isolation happens before assembly, at dispatch time

        # --- Assemble the final context ---
        system_tokens = estimate_tokens(system_prompt)
        query_tokens = estimate_tokens(query)

        context = f"<system>\n{system_prompt}\n</system>\n\n"

        if memory_block:
            context += f"{memory_block}\n\n"

        if compressed_history:
            context += "<history>\n"
            context += "\n".join(compressed_history)
            context += "\n</history>\n\n"

        if retrieved_docs:
            context += "<retrieved_context>\n"
            for doc_id, content, score in retrieved_docs:
                # Include a relevance header
                context += f"  --- [{score:.2f}] {doc_id} ---\n"
                context += f"  {content[:500]}...\n\n"  # truncate individual docs too
            context += "</retrieved_context>\n\n"

        context += f"<query>{query}</query>"

        # Token accounting
        total_tokens = estimate_tokens(context)
        breakdown = {
            "system": system_tokens,
            "memory": memory_tokens,
            "history": history_tokens,
            "retrieved": retrieved_tokens,
            "query": query_tokens,
            "total": total_tokens,
            "available": self.available,
            "utilization": total_tokens / self.available * 100,
        }

        return {
            "context": context,
            "breakdown": breakdown,
            "strategies_used": {
                "write": memory_tokens > 0,
                "select": len(retrieved_docs) > 0,
                "compress": len(compressed_history) < len(history),
                "isolate": len(self.sub_agent_mgr.sub_agents) > 0,
            },
            "docs_selected": len(retrieved_docs),
            "history_compressed": f"{len(compressed_history)}/{len(history)} messages",
        }

    def print_budget_report(self, breakdown):
        """Print a detailed budget utilization report."""
        print("=" * 60)
        print("  CONTEXT ENGINE — BUDGET REPORT")
        print("=" * 60)
        print(f"\n  Available budget:    {breakdown['available']:>8,} tokens")
        print(f"  Total used:          {breakdown['total']:>8,} tokens")
        print(f"  Utilization:         {breakdown['utilization']:>7.1f}%")
        print(f"\n  Breakdown:")
        for key in ['system', 'memory', 'history', 'retrieved', 'query']:
            pct = breakdown[key] / breakdown['available'] * 100
            bar_len = int(pct / 2)
            bar = "█" * bar_len + "░" * (50 - bar_len)
            print(f"    {key:<12s} {breakdown[key]:>7,} tokens ({pct:>5.1f}%) {bar}")
        print("=" * 60)

In [ ]:
# Build the complete engine and populate it with our simulated data

engine = ContextEngine(max_tokens=128000, reserved_for_output=35000)

# Strategy 1: Populate memory
for key, value, _, tags in [
    ("build", "pytest tests/ -v --cov", "project", ["test"]),
    ("formatter", "Black, 88 char line length", "project", ["style"]),
    ("database", "PostgreSQL 15, alembic migrations", "project", ["db"]),
    ("deploy", "docker-compose up --build", "project", ["deploy"]),
    ("naming", "snake_case for Python, camelCase for JS", "style", ["convention"]),
]:
    engine.memory.save(key, value, category="project")

# Strategy 2: Register documents in selector
for filepath, summary, content, tags in files:
    engine.selector.register(filepath, summary, content, tags)

# Now assemble context for a query
query = "Fix the JWT token refresh bug in the authentication module"
system_prompt = "You are a senior software engineer. Help debug and fix issues in the codebase. Always explain your reasoning step by step."

result = engine.assemble(
    query=query,
    system_prompt=system_prompt,
    history=conversation,
    doc_query="authentication JWT token refresh"
)

# Print the budget report
engine.print_budget_report(result["breakdown"])

print(f"\n  Strategies active:")
for strategy, used in result["strategies_used"].items():
    status = "ON" if used else "OFF"
    print(f"    {strategy:<10s}: {status}")
print(f"\n  Documents selected: {result['docs_selected']}")
print(f"  History: {result['history_compressed']}")

In [ ]:
#@title 🎧 Listen: Strategy Comparison
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/20_strategy_comparison.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 6. Putting It All Together — The Strategy Comparison Experiment

Now for the grand finale: we run the same query through four configurations and compare token usage.

In [ ]:
# Configuration 1: No strategies (raw context dump)
raw_context_parts = [
    system_prompt,
    "\n".join(project_memory),
    "\n".join(conversation),
    "\n".join(selector.fetch(fid) for fid in selector.store),
    query,
]
raw_tokens = sum(estimate_tokens(part) for part in raw_context_parts)

# Configuration 2: Write only
write_only_parts = [
    system_prompt,
    engine.memory.format_for_context(),
    "\n".join(conversation),
    "\n".join(selector.fetch(fid) for fid in selector.store),
    query,
]
write_only_tokens = sum(estimate_tokens(part) for part in write_only_parts)

# Configuration 3: Write + Select
selected_for_query = engine.selector.select("authentication JWT token refresh", top_k=3)
selected_ids_q = [doc_id for doc_id, _, _ in selected_for_query]
write_select_parts = [
    system_prompt,
    engine.memory.format_for_context(),
    "\n".join(conversation),
    "\n".join(selector.fetch(fid) for fid in selected_ids_q if selector.fetch(fid)),
    query,
]
write_select_tokens = sum(estimate_tokens(part) for part in write_select_parts)

# Configuration 4: Write + Select + Compress
cleaned, _ = engine.compressor.clear_tool_results(conversation)
compressed, comp_tokens = engine.compressor.compress_history(cleaned, int(engine.available * 0.3))
write_select_compress_parts = [
    system_prompt,
    engine.memory.format_for_context(),
    "\n".join(compressed),
    "\n".join(selector.fetch(fid) for fid in selected_ids_q if selector.fetch(fid)),
    query,
]
write_select_compress_tokens = sum(estimate_tokens(part) for part in write_select_compress_parts)

# Configuration 5: All four strategies (Write + Select + Compress + Isolate)
# For isolate, replace full docs with sub-agent summaries
sub_tasks = [
    {'name': 'auth', 'description': 'Auth analysis', 'doc_ids': selected_ids_q}
]
sub_docs = {fid: selector.fetch(fid) for fid in selected_ids_q if selector.fetch(fid)}
sub_result = dispatch_to_subagents_solution(sub_tasks, sub_docs, max_summary_tokens=400)
sub_summary_text = "\n".join(s for _, s in sub_result['summaries'])

all_four_parts = [
    system_prompt,
    engine.memory.format_for_context(),
    "\n".join(compressed),
    sub_summary_text,
    query,
]
all_four_tokens = sum(estimate_tokens(part) for part in all_four_parts)

# Results
configs = [
    ("No strategies\n(raw dump)", raw_tokens),
    ("Write\nonly", write_only_tokens),
    ("Write +\nSelect", write_select_tokens),
    ("Write + Select\n+ Compress", write_select_compress_tokens),
    ("All four\nstrategies", all_four_tokens),
]

print("=" * 65)
print("  STRATEGY COMPARISON — TOKEN USAGE")
print("=" * 65)
for name, tokens in configs:
    pct_of_raw = tokens / raw_tokens * 100
    savings = raw_tokens - tokens
    print(f"  {name.replace(chr(10), ' '):<30s} {tokens:>8,} tokens  "
          f"({pct_of_raw:>5.1f}% of raw)  saves {savings:>6,}")
print("=" * 65)

In [ ]:
#@title 🎧 Listen: Comparison Barchart
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/21_comparison_barchart.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### 📊 Visualization Checkpoint 2: Strategy Comparison Bar Chart

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Bar chart of token usage
names = [c[0] for c in configs]
tokens_list = [c[1] for c in configs]
colors_bar = ['#E53935', '#FF7043', '#FFA726', '#66BB6A', '#26A69A']

bars = axes[0].bar(range(len(names)), tokens_list, color=colors_bar,
                    edgecolor='white', linewidth=2, width=0.65)

# Add budget line
axes[0].axhline(y=engine.available, color='red', linestyle='--', linewidth=2,
                 label=f'Budget ({engine.available:,})')

for bar, tok in zip(bars, tokens_list):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 200,
                 f'{tok:,}', ha='center', fontsize=9, fontweight='bold')

axes[0].set_xticks(range(len(names)))
axes[0].set_xticklabels(names, fontsize=9)
axes[0].set_ylabel('Tokens', fontsize=12)
axes[0].set_title('Token Usage by Configuration', fontsize=13, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3, axis='y')

# Savings percentage chart
savings_pcts = [(1 - tok / raw_tokens) * 100 for tok in tokens_list]
bars2 = axes[1].bar(range(len(names)), savings_pcts, color=colors_bar,
                     edgecolor='white', linewidth=2, width=0.65)

for bar, pct in zip(bars2, savings_pcts):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                 f'{pct:.0f}%', ha='center', fontsize=10, fontweight='bold')

axes[1].set_xticks(range(len(names)))
axes[1].set_xticklabels(names, fontsize=9)
axes[1].set_ylabel('Token Savings (%)', fontsize=12)
axes[1].set_title('Cumulative Savings', fontsize=13, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')

plt.suptitle('Context Engineering: Adding Strategies One by One',
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 Listen: Pie Charts
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/22_pie_charts.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### 📊 Visualization Checkpoint 3: Context Composition — Before and After

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# PIE 1: Before (no strategies)
before_sizes = [
    estimate_tokens(system_prompt),
    sum(estimate_tokens(m) for m in project_memory),
    sum(estimate_tokens(m) for m in conversation),
    sum(estimate_tokens(selector.fetch(fid)) for fid in selector.store),
    estimate_tokens(query),
]
before_labels = ['System\nPrompt', 'Memory\n(repeated)', 'Full\nHistory',
                 'All Docs\n(eager)', 'Query']
before_colors = ['#42A5F5', '#66BB6A', '#FFA726', '#EF5350', '#AB47BC']

wedges1, texts1, autotexts1 = axes[0].pie(
    before_sizes, labels=before_labels, colors=before_colors,
    autopct='%1.0f%%', pctdistance=0.75, startangle=90,
    textprops={'fontsize': 9}
)
axes[0].set_title(f'Before: No Strategies\n({sum(before_sizes):,} tokens)',
                   fontsize=13, fontweight='bold')

# PIE 2: After (all four strategies)
after_system = estimate_tokens(system_prompt)
after_memory = engine.memory.get_token_cost()
after_history = comp_tokens
after_docs = estimate_tokens(sub_summary_text)
after_query = estimate_tokens(query)

after_sizes = [after_system, after_memory, after_history, after_docs, after_query]
after_labels = ['System\nPrompt', 'Memory\n(file)', 'Compressed\nHistory',
                'Sub-agent\nSummaries', 'Query']

wedges2, texts2, autotexts2 = axes[1].pie(
    after_sizes, labels=after_labels, colors=before_colors,
    autopct='%1.0f%%', pctdistance=0.75, startangle=90,
    textprops={'fontsize': 9}
)
axes[1].set_title(f'After: All Four Strategies\n({sum(after_sizes):,} tokens)',
                   fontsize=13, fontweight='bold')

plt.suptitle('Context Composition: Before vs After Optimization',
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

reduction = (1 - sum(after_sizes) / sum(before_sizes)) * 100
print(f"Total reduction: {sum(before_sizes):,} -> {sum(after_sizes):,} tokens ({reduction:.0f}% savings)")

In [ ]:
#@title 🎧 Listen: Results
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/23_results.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 7. Results — The Assembled Context

Let us see the final assembled context itself, properly formatted.

In [ ]:
# Run the full engine and display the output
final_result = engine.assemble(
    query="Fix the JWT token refresh bug in the authentication module",
    system_prompt="You are a senior software engineer. Help debug and fix issues.",
    history=conversation,
    doc_query="authentication JWT token refresh"
)

print("=" * 70)
print("  FINAL ASSEMBLED CONTEXT (truncated for display)")
print("=" * 70)

context_lines = final_result["context"].split("\n")
# Show first 40 lines and last 10 lines
display_lines = context_lines[:40]
if len(context_lines) > 50:
    display_lines.append(f"\n  ... [{len(context_lines) - 50} lines omitted] ...\n")
    display_lines.extend(context_lines[-10:])

for line in display_lines:
    print(f"  {line}")

print("\n" + "=" * 70)

In [ ]:
# Final budget report with color-coded utilization

engine.print_budget_report(final_result["breakdown"])

# Strategy summary
print("\n  Strategy Effectiveness:")
print("  ─────────────────────────────────────────")

strategy_data = [
    ("Write",    "Persistent memory",        "Avoid repeating project facts",
     memory.get_token_cost(), "tokens in memory file"),
    ("Select",   "Just-in-time retrieval",   f"Loaded {final_result['docs_selected']} of {len(selector.store)} docs",
     final_result["breakdown"]["retrieved"], "tokens retrieved"),
    ("Compress", "History summarization",     final_result["history_compressed"],
     final_result["breakdown"]["history"], "tokens in history"),
    ("Isolate",  "Sub-agent delegation",     "Condensed summaries from focused agents",
     estimate_tokens(sub_summary_text), "tokens in summaries"),
]

for name, mechanism, detail, tokens, unit in strategy_data:
    status = "ACTIVE" if final_result["strategies_used"].get(name.lower(), False) else "READY"
    print(f"  [{status:>6s}] {name:<10s} | {mechanism:<25s} | {tokens:>6,} {unit}")
    print(f"           {'':<10s} | {detail}")

In [ ]:
#@title 🎧 Listen: Waterfall
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/24_waterfall.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

In [ ]:
# Grand summary visualization: strategy impact waterfall chart

fig, ax = plt.subplots(figsize=(14, 7))

# Waterfall chart showing how each strategy reduces tokens
labels = ['Raw\n(no strategies)', 'After\nWrite', 'After\nSelect',
          'After\nCompress', 'After\nIsolate', 'Final\nContext']
values = [raw_tokens, write_only_tokens, write_select_tokens,
          write_select_compress_tokens, all_four_tokens, all_four_tokens]

# Compute the changes
changes = [0]  # first bar starts from 0
for i in range(1, len(values) - 1):
    changes.append(values[i] - values[i-1])
changes.append(0)  # last bar is the final total

# Draw waterfall
bottoms = [0] * len(labels)
bar_colors = ['#E53935']  # first bar red (full)

for i in range(1, len(values) - 1):
    bottoms[i] = values[i]
    bar_colors.append('#66BB6A')  # green for savings

bottoms[-1] = 0
bar_colors.append('#2196F3')  # blue for final

# Draw the bars
bar_heights = [raw_tokens]  # first bar
for i in range(1, len(values) - 1):
    bar_heights.append(values[i-1] - values[i])  # savings
bar_heights.append(all_four_tokens)  # final bar

# Recompute bottoms for waterfall
cumulative = raw_tokens
waterfall_bottoms = [0]
waterfall_heights = [raw_tokens]

for i in range(1, len(values) - 1):
    saving = values[i-1] - values[i]
    cumulative -= saving
    waterfall_bottoms.append(cumulative)
    waterfall_heights.append(saving)

waterfall_bottoms.append(0)
waterfall_heights.append(all_four_tokens)

bars = ax.bar(range(len(labels)), waterfall_heights, bottom=waterfall_bottoms,
              color=bar_colors, edgecolor='white', linewidth=2, width=0.6)

# Add value labels
for i, (bar, height, bottom) in enumerate(zip(bars, waterfall_heights, waterfall_bottoms)):
    if i == 0:
        ax.text(bar.get_x() + bar.get_width()/2, raw_tokens + 200,
                f'{raw_tokens:,}', ha='center', fontsize=10, fontweight='bold')
    elif i == len(labels) - 1:
        ax.text(bar.get_x() + bar.get_width()/2, all_four_tokens + 200,
                f'{all_four_tokens:,}', ha='center', fontsize=10,
                fontweight='bold', color='#1565C0')
    else:
        ax.text(bar.get_x() + bar.get_width()/2, bottom + height/2,
                f'-{height:,}', ha='center', fontsize=10,
                fontweight='bold', color='white')

# Add connector lines
for i in range(len(labels) - 2):
    y = waterfall_bottoms[i+1] + waterfall_heights[i+1] if i > 0 else raw_tokens
    ax.plot([i + 0.3, i + 0.7], [waterfall_bottoms[i+1] + waterfall_heights[i+1]] * 2,
            color='gray', linewidth=1, linestyle='--', alpha=0.5)

# Budget line
ax.axhline(y=engine.available, color='red', linestyle='--', linewidth=2,
           alpha=0.7, label=f'Budget limit ({engine.available:,})')

ax.set_xticks(range(len(labels)))
ax.set_xticklabels(labels, fontsize=10)
ax.set_ylabel('Tokens', fontsize=12)
ax.set_title('Context Engineering Waterfall: How Each Strategy Reduces Tokens',
             fontsize=14, fontweight='bold')
ax.legend(fontsize=11, loc='upper right')
ax.grid(True, alpha=0.2, axis='y')

# Custom legend
legend_elements = [
    mpatches.Patch(color='#E53935', label='Raw context (no optimization)'),
    mpatches.Patch(color='#66BB6A', label='Tokens saved by strategy'),
    mpatches.Patch(color='#2196F3', label='Final optimized context'),
]
ax.legend(handles=legend_elements, fontsize=10, loc='upper right')

plt.tight_layout()
plt.show()

print(f"\nTotal reduction: {raw_tokens:,} -> {all_four_tokens:,} tokens")
print(f"Savings: {raw_tokens - all_four_tokens:,} tokens ({(1 - all_four_tokens/raw_tokens)*100:.0f}%)")

In [ ]:
#@title 🎧 Listen: Dashboard
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/25_dashboard.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

In [ ]:
# Final comprehensive dashboard

fig = plt.figure(figsize=(18, 10))

# Layout: 2x3 grid
ax1 = fig.add_subplot(2, 3, 1)  # Strategy savings bars
ax2 = fig.add_subplot(2, 3, 2)  # Before pie
ax3 = fig.add_subplot(2, 3, 3)  # After pie
ax4 = fig.add_subplot(2, 1, 2)  # Budget utilization timeline

# 1. Individual strategy savings
individual_savings = {
    'Write': raw_tokens - write_only_tokens,
    'Select': write_only_tokens - write_select_tokens,
    'Compress': write_select_tokens - write_select_compress_tokens,
    'Isolate': write_select_compress_tokens - all_four_tokens,
}
strat_names = list(individual_savings.keys())
strat_values = list(individual_savings.values())
strat_colors = ['#4CAF50', '#2196F3', '#FF9800', '#9C27B0']

bars_ind = ax1.barh(strat_names, strat_values, color=strat_colors,
                     edgecolor='white', linewidth=2, height=0.5)
for bar, val in zip(bars_ind, strat_values):
    ax1.text(bar.get_width() + 50, bar.get_y() + bar.get_height()/2,
             f'{val:,}', va='center', fontsize=10, fontweight='bold')
ax1.set_xlabel('Tokens Saved', fontsize=11)
ax1.set_title('Savings per Strategy', fontsize=12, fontweight='bold')
ax1.grid(True, alpha=0.3, axis='x')

# 2. Before pie
ax2.pie(before_sizes, labels=['System', 'Memory', 'History', 'Docs', 'Query'],
        colors=before_colors, autopct='%1.0f%%', startangle=90,
        textprops={'fontsize': 8})
ax2.set_title(f'Before\n({sum(before_sizes):,} tokens)', fontsize=12, fontweight='bold')

# 3. After pie
ax3.pie(after_sizes, labels=['System', 'Memory', 'History', 'Docs', 'Query'],
        colors=before_colors, autopct='%1.0f%%', startangle=90,
        textprops={'fontsize': 8})
ax3.set_title(f'After\n({sum(after_sizes):,} tokens)', fontsize=12, fontweight='bold')

# 4. Budget utilization bar (horizontal stacked)
sections = ['System', 'Memory', 'History', 'Retrieved', 'Query', 'Free']
section_tokens = [
    final_result["breakdown"]["system"],
    final_result["breakdown"]["memory"],
    final_result["breakdown"]["history"],
    final_result["breakdown"]["retrieved"],
    final_result["breakdown"]["query"],
    engine.available - final_result["breakdown"]["total"],
]
section_colors = ['#42A5F5', '#66BB6A', '#FFA726', '#EF5350', '#AB47BC', '#E0E0E0']

left = 0
for name, tokens, color in zip(sections, section_tokens, section_colors):
    width = tokens / engine.available * 100
    ax4.barh(0, width, left=left, color=color, edgecolor='white',
             linewidth=1.5, height=0.4, label=f'{name} ({tokens:,})')
    if width > 4:
        ax4.text(left + width/2, 0, f'{name}\n{width:.0f}%',
                 ha='center', va='center', fontsize=9, fontweight='bold')
    left += width

ax4.set_xlim(0, 100)
ax4.set_yticks([])
ax4.set_xlabel('Budget Utilization (%)', fontsize=11)
ax4.set_title('Final Context Budget Allocation', fontsize=12, fontweight='bold')
ax4.legend(fontsize=8, loc='upper right', ncol=3)

plt.suptitle('Complete Context Engine — Dashboard',
             fontsize=15, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 Listen: Reflection
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/26_reflection.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 8. Reflection and Next Steps

### 💡 Key Takeaways

1. **Write** is the simplest strategy but prevents costly repetition. Claude Code's `CLAUDE.md` is a production example. Even saving 5 project facts eliminates hundreds of tokens of redundant conversation.

2. **Select** delivers the largest single-strategy savings. Loading 3 files instead of 15 cuts document tokens by 80%. The key insight: maintain a cheap index, fetch content just-in-time.

3. **Compress** is essential for long sessions. Conversation history grows linearly with time. TF-IDF extractive summarization keeps the signal (architectural decisions, bug reports) and discards the noise (greetings, confirmations). Clearing tool results alone can save thousands of tokens.

4. **Isolate** shines on parallelizable tasks. Instead of one agent drowning in 15 files, three sub-agents each process 2-3 files and return compact summaries. Anthropic's multi-agent researcher demonstrated that isolation improves both quality and efficiency.

5. **The strategies compound.** Each strategy targets a different part of the context budget. Write handles memory, Select handles documents, Compress handles history, Isolate handles complexity. Together, they can reduce context usage by 60-80%.

### 🤔 Reflection Questions

1. In your own projects, which strategy would give you the **biggest immediate win**? Why?

2. Claude Code auto-compacts at 95% capacity. Why 95% and not 50%? What is the tradeoff between compacting early vs late?

3. The Isolate strategy works best for "read-only" tasks (research, analysis). Why might it be dangerous for "write" tasks (code editing, file creation)?

4. Could you build a context engine that **learns** the optimal budget allocation over time? What signal would you use to tune the percentages?

In [ ]:
# Quick self-check: revisit your predictions from Section 2

print("Revisit Your Predictions:")
print("=" * 55)
print()
print("1. Which strategy saves the MOST tokens?")
print("   Answer: SELECT (just-in-time retrieval)")
print("   Loading 3 of 15 files saves ~80% of document tokens.")
print()
print("2. Which strategy is CHEAPEST to implement?")
print("   Answer: WRITE (persistent memory)")
print("   It is just JSON read/write — no ML, no embeddings.")
print()
print("3. Do you need all four strategies?")
print("   Answer: Each targets a different budget section.")
print("   You COULD survive with just Select + Compress,")
print("   but all four together give the best results.")
print()
print("Did your predictions match? If not, that is learning!")

In [ ]:
#@title 🎧 Listen: What You Built
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/27_what_you_built.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### What You Built

In this notebook, you built a complete context engineering system:

| Component | Class | Strategy | Key Method |
|-----------|-------|----------|------------|
| Persistent memory | `MemoryStore` | Write | `save()`, `search()`, `format_for_context()` |
| Document retrieval | `DocumentSelector` | Select | `select()`, `fetch()` |
| History compression | `ContextCompressor` | Compress | `compress_history()`, `clear_tool_results()` |
| Sub-agent dispatch | `SubAgentManager` | Isolate | `dispatch()`, `simulate_execution()` |
| Full engine | `ContextEngine` | All four | `assemble()`, `print_budget_report()` |

### The Central Lesson

Context engineering is not about having a bigger window. It is about using the window you have **intelligently**. The four strategies — Write, Select, Compress, Isolate — are the building blocks that every frontier agent system uses, from Claude Code to Cursor to Devin.

The best context is not the most context. It is the **right** context, at the **right** time, in the **right** amount.

In [ ]:
print("=" * 60)
print("  BUILDING A COMPLETE CONTEXT ENGINE — KEY TAKEAWAYS")
print("=" * 60)
print()
print("  1. WRITE:    Persist facts outside the window")
print("               Save once, read on demand, never repeat")
print()
print("  2. SELECT:   Retrieve just-in-time")
print("               Cheap index + lazy fetch = massive savings")
print()
print("  3. COMPRESS: Summarize, don't truncate")
print("               TF-IDF extracts signal, drops noise")
print()
print("  4. ISOLATE:  Divide and conquer")
print("               Sub-agents with focused context > one bloated agent")
print()
print("  Combined: 60-80% token reduction with no quality loss.")
print()
print("=" * 60)
print("  Great work! You have completed the Context Engine notebook.")
print("=" * 60)